# LLM & LangChain 실습: Ollama를 활용한 자연어 처리

## 학습 목표
이번 실습을 통해 다음을 학습합니다:
- Ollama를 활용한 로컬 LLM 모델 사용법
- LangChain을 이용한 체인 구성
- 프롬프트 엔지니어링 기법
- 대화형 AI 시스템 구축

## 1. LLM & LangChain 소개

### 1.1 LLM(Large Language Model)이란?
- **정의**: 대규모 텍스트 데이터로 훈련된 신경망 기반 언어 모델
- **특징**: 자연어 이해, 생성, 번역, 요약 등 다양한 NLP 태스크 수행 가능
- **대표모델**: GPT, LLaMA, Claude 등

### 1.2 LangChain이란?
- **정의**: LLM을 활용한 애플리케이션 개발을 위한 프레임워크
- **특징**: 프롬프트 관리, 체인 구성, 메모리 관리 등 편리한 기능 제공
- **구성요소**: Prompts, LLMs, Chains, Memory, Agents 등

### 1.3 Ollama란?
- **정의**: 로컬에서 LLM을 실행할 수 있게 해주는 도구
- **장점**: 오프라인 사용 가능, 데이터 프라이버시 보장
- **지원모델**: LLaMA, Mistral, CodeLlama 등

## 2. 기본 LLM 사용법

### 2.1 간단한 질문-답변

-  **LLM 연결**  `Ollama(model="llama3.1")`을 통해 로컬에서 실행 중인 LLaMA 3.1 모델에 연결할 수 있습니다.

In [1]:
from langchain_community.llms import Ollama

# Ollama 연결 (localhost:11434에서 실행 중인 Ollama에 연결)
llm = Ollama(model="llama3.1")

# chain 실행
response = llm.invoke("세계에서 가장 붐비는 항공노선은 어디인가?")

# 결과 출력
print(response)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8660\323340537.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")


다음은 세계에서 가장 붐비는 항공 노선 10여 개이다. 

1. 뉴욕(런던) - 뉴욕(파리)

이번 주말, 이 루트의 승객 수는 11만6천 명으로 전주간보다 4%가량 늘었다.

2. 뉴욕(런던) - 뉴욕(로마)

이번 주말, 승객 수는 9만9천 명이었다.  

3. 파리(도쿄) - 런던(도쿄)

파리와 도쿄를 오가는 국제 노선은 지난해보다 20%나 늘었다. 

4. 뉴욕(로마) - 뉴욕(베이징)

지난주부터 이번주까지 승객 수는 전주간에 비해 6%나 줄어들었지만 여전히 세계에서 가장 붐비는 항공 노선이다.

5. 파리(신자카르파터) - 로마(도쿄)

이번 주말, 승객은 전주간보다 10% 더많았다.

6. 뉴욕(로마) - 파리

7. 베이징(뉴욕) - 도쿄

8. 베이징(런던) - 런던(신자카르파터)

9. 로마(도쿄) - 신자카르파터

10. 런던(신자카르파터) - 뉴욕 

항공업계의 여정 승객 수 조사 결과에 따르면, 전 세계적으로 2023년 5월 항공여객수는 전년 동월 대비 16%가량 증가했다.


### 2.2 Temperature 설정과 스트리밍 출력
- **Temperature**: 모델의 창의성을 조절하는 매개변수 (0~1)
- **스트리밍**: `.stream()` 메서드로 실시간 응답 확인 가능
- **프롬프트 엔지니어링**: 구체적인 형식과 출처 요구로 답변 품질 향상

In [2]:
import sys

llm = Ollama(temperature=0, model="llama3.1")

question = "세계에서 가장 붐비는 항공노선 4개를 알려줘"

summary_query = f"""
아래 질문에 대한 대답을 할 때 IATA 자료를 참고해. 
대답은 아래 형식으로 만들어줘. 마지막에는 출처를 적어줘.

형식:
1. 대답 1
2. 대답 2

질문: {question}
"""
# 스트리밍 출력
for chunk in llm.stream(summary_query):
    sys.stdout.write(chunk)
    sys.stdout.flush()
    
print()  # 마지막에 줄바꿈을 추가

1. 도쿄(나리타) - 오사카 노선 (일일 평균 승객 수 : 약 13,000명)
2. 뉴욕 존 F. 케네디 공항 - 로스앤젤레스 국제공항 노선 (일일 평균 승객 수 : 약 12,000명)
3. 도쿄(나리타) - 홍콩 노선 (일일 평균 승객 수 : 약 10,000명)
4. 서울(인천) - 도쿄(나리타) 노선 (일일 평균 승객 수 : 약 9,000명)

출처: IATA (국제 항공 운송 협회)


## 3. LangChain 체인 구성

### 3.1 기본 체인 구성 (LCEL)

- **LCEL**: `|` 연산자로 체인을 간단하게 연결
- **ChatPromptTemplate**: 역할 기반 프롬프트 설정
- **StrOutputParser**: 출력 결과를 문자열로 파싱

In [3]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(
    "당신은 유능한 기상학자입니다. 다음 질문에 답해주세요. <질문>: {question}"
)

# 2. Ollama 모델 초기화
llm = Ollama(model="llama3.1")

# 3. 스트림 출력 파서 생성
class CustomStreamOutputParser(StrOutputParser):
    def parse(self, text):
        return text
    
output_parser = CustomStreamOutputParser()

# 4. chain 연결 (LCEL)
chain = prompt | llm | output_parser

# 5. chain 실행 및 결과 출력
for chunk in chain.stream({"question": "크기에 따른 태풍의 분류 방법을 알려주세요"}):
    print(chunk, end="", flush=True)

태풍의 크기에 따라 다음과 같은 분류가 있습니다.

- 열대 저기압(Tropical Depression): 중심기압 30mbar 미만, 최대풍속 55km/h 이하
- 열대低氣壓은 태풍의 초기 단계입니다. 중심기압이 약간 낮고 풍속도 조용한 상태입니다.
- 열대 강우(Basin T Storm): 중심기압 28.9mbar 이상, 최대풍속 56~119km/h 이하
- 열대 폭풍은 태풍이 성장하기 시작하는 단계입니다. 풍속이 약간 높아졌고 강풍영역도 확대되었습니다.
- 태풍(Typhoon): 중심기압 27.1mbar 이상, 최대풍속 120km/h 이상
- 태풍은 가장 높은 분류로써 최대로 성장한 상태입니다. 풍속과 강풍영역이 크고 강력합니다.

참고로, 이러한 분류는 한국 기상청의 기준에 따라 분류가 되며, 세계 기상기구(WMO)에서는 약간의 차이가 있을 수 있습니다.

## 실습

아래 코드에서 `system` 메시지 부분을 완성해보세요.

**목표**: 사용자가 계절을 입력하면, 그 계절에 맞는 국내 여행지 3곳을 추천하고 이유를 설명하는 AI를 만들어보세요.

**예상 출력**:
1. 제주도 - 화이트 크리스마스, 추운 겨울이 시작되기 전의 마지막 여름의 아름다움으로 시원한 봄날씨와 함께 보석처럼 빛나는 성을 경험할 수 있습니다. 
2. 경복궁 - 봄이 오면 벚꽃이 피는 경복궁은 분위기 좋은 장소로, 역사와 문화를 느낄 수 있는 곳입니다.
3. 설악산 - 봄의 아경과 함께 아름다운 지리산의 신록들을 산책하며 자연을 감상할 수 있습니다.

In [ ]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser



# 1. 프롬프트 명시적으로 형식 지시 포함
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "당신은 여행 전문가입니다."



     
                                ), # 괄호 안에 프롬포트를 요구사항에 맞게 한글로 작성
    ("user", "{question}")
])






# 2. Ollama 모델 초기화
llm = Ollama(model="llama3.1")

# 3. 출력 파서 설정
output_parser = StrOutputParser()

# 4. 체인 구성
chain = prompt | llm | output_parser

# 5. 실행 예시
question = "봄에 어디로 여행 가면 좋을까?"
response = chain.invoke({"question": question})

print(response)


봄에는 한국의 정취를 느끼는 곳으로 여행하기 좋은 장소가 여러곳 있습니다. 한남동(서울), 경복궁 근처, 영천, 남해 등이 그 예입니다.

한남동은 봄이되면서 가을보다도 더 아름다운 풍경을 감상할 수 있는 곳입니다. 경복궁 근처는 역사적인 느낌과 함께 가을의 선선한 기운이 느껴집니다. 영천에는 정원에 가득한 아름다운 절이 있고 남해에서는 바다의 휴식처로 인기있는 곳입니다.


: 

## 4. 대화형 AI 시스템 구축

### 4.1 대화 기록을 유지하는 채팅봇

- **MessagesPlaceholder**: 동적으로 대화 기록을 삽입
- **대화 기록 관리**: 리스트에 (역할, 메시지) 튜플로 저장
- **컨텍스트 유지**: 이전 대화를 참고하여 일관된 응답 생성

In [ ]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# 1. 대화내용 저장을 위한 ChatPromptTemplate 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 기상학자입니다. 답변은 200자 이내로 하세요."),
    # 1-1. 프롬프트에 대화 기록용 chat_history 추가
    MessagesPlaceholder("chat_history"), 
    ("user", "{question}")
])

# 2. Ollama 모델 초기화
llm = Ollama(model="llama3.1")

# 3. 스트림 출력 파서 생성
class CustomStreamOutputParser(StrOutputParser):
    def parse(self, text):
        return text
    
output_parser = CustomStreamOutputParser()

# 4. chain 연결 (LCEL)
chain = prompt | llm | output_parser

# 5. 채팅 기록 초기화
chat_history = []

# 6. chain 실행 및 결과 출력을 반복
while True:
    # 6-1. 사용자의 입력을 기다림
    user_input = input("\n\n당신: ")
    if user_input == "끝":
        break
        
    # 6-2. 체인을 실행하고 결과를 stream 형태로 출력
    result = ""
    for chunk in chain.stream({"question": user_input, "chat_history": chat_history}):
        print(chunk, end="", flush=True)
        result += chunk
        
    # 6-3. 채팅 기록 업데이트
    chat_history.append(("user", user_input))
    chat_history.append(("assistant", result))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12236\1438323756.py:14: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")
